# Greedy Agent function from organizers

In [ ]:
%%writefile greedy_agent.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col


def greedy_agent(obs_dict, config_dict):
    """This agent always moves toward observation.food[0] but does not take advantage of board wrapping"""
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    food = observation.food[0]
    food_row, food_column = row_col(food, configuration.columns)

    if food_row > player_row:
        return Action.SOUTH.name
    if food_row < player_row:
        return Action.NORTH.name
    if food_column > player_column:
        return Action.EAST.name
    return Action.WEST.name

# Custom Greedy Agent

Here some modifications were done:
* Find the closest food item and go to it
* Try to move in each of 4 possible directions and choose the optimal one
* Before making a move check wheather it will be counteracting with other geese bodies

In [ ]:
%%writefile agent.py

from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col, min_distance
from itertools import chain
import numpy as np

# Presettings for directions
last_direction = ''

# Create dictionary of opposite directions 
opposite_direction = {Action.SOUTH.name: Action.NORTH.name,
                      Action.NORTH.name: Action.SOUTH.name,
                      Action.EAST.name: Action.WEST.name,
                      Action.WEST.name: Action.EAST.name,
                     '': ''}


def possible_moves(position, rows, columns):
    """Generate all possible moves from position"""
    
    # Current position decoding
    row, column = row_col(position, columns)
    
    # Possible moves
    dx = [-1, 0, 1, 0]
    dy = [0, -1, 0, 1]

    # Try to move in each direction
    possible_moves_ = []
    for x, y in zip(dx, dy):
        new_row = (row + x + rows) % rows
        new_column = (column + y + columns) % columns
        possible_moves_.append(new_row * columns + new_column)

    return possible_moves_
    
    
def make_move(from_, to_, rows, columns):
    """Returns direction in which we should go if we want to move from from_ position to to_ position"""
    
    # Convert positions to row-column
    from_row, from_column = row_col(from_, columns)
    to_row, to_column = row_col(to_, columns)

    # Count differences between from and to
    row_diff = from_row - to_row
    column_diff = from_column - to_column
    
    # Edge cases (if we want to go out of board)
    if row_diff == rows-1:
        return Action.SOUTH.name
    elif row_diff == -(rows-1):
        return Action.NORTH.name
    elif column_diff == columns-1:
        return Action.EAST.name
    elif column_diff == -(columns-1):
        return Action.WEST.name
    
    # Most likely case of move
    if to_row > from_row:
        return Action.SOUTH.name
    if to_row < from_row:
        return Action.NORTH.name
    if to_column > from_column:
        return Action.EAST.name
    return Action.WEST.name

    
def distance(pos1, pos2, columns):
    """Compute distance between two positions"""
    
    # Convert pos to row-column format
    pos1_row, pos1_column = row_col(pos1, columns)
    pos2_row, pos2_column = row_col(pos2, columns)
    
    # Compute distance
    distance = abs(pos1_row - pos2_row) + abs(pos1_column - pos2_column)
    
    return distance


def get_closest_food_index(player, food, columns):
    """Finds the closest food index between goose and array of foods."""
    
    # Compute distance from player position to all food items
    distances = [
        distance(player, food_position, columns)
        for food_position in food
    ]
    
    # Find closest food index
    closest_index = np.argmin(distances)
    
    return closest_index


def get_goose_next_placement(goose, observation, configuration):
    """Find goose placement on the next step."""
    
    if not goose:
        return set()
    
    # Retrieve basic information for goose
    goose_head = goose[0]
    rows, columns = configuration.rows, configuration.columns
    
    # Get all possible moves
    possible_moves_ = possible_moves(goose_head, rows, columns)
    
    # Placement of goose is equal to current placement and one cell towards food
    next_move_placement = set(goose + possible_moves_)

    return next_move_placement


def custom_agent(obs_dict, config_dict):
    """Customized version of organizers agent with modifications in choosing food item and in choosing next move."""
    
    # Create python objects of game
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    rows, columns = configuration.rows, configuration.columns
    
    # Get basic information about ourself
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, columns)

    # Get information about geese positions of other players
    geese_positions = observation.geese
    geese_positions.pop(player_index)
    
    # Find positions which will be used by others geese in the next move
    dangerous_geese_positions = [get_goose_next_placement(goose, observation, configuration)
                                 for goose in geese_positions]
    dangerous_geese_positions = list(chain.from_iterable(dangerous_geese_positions))
    
    # Find closest food and get its position
    closest_food_index = get_closest_food_index(player_head, observation.food, columns)
    food = observation.food[closest_food_index]
    
    # Get all possible moves
    possible_moves_ = possible_moves(player_head, rows, columns)

    # Get distances to the food and sort moves by their distance to food
    moves_distances = [distance(move, food, columns) for move in possible_moves_]
    preferred_moves = [possible_moves_[idx] for idx in np.argsort(moves_distances)]
    
    # Default move direction
    new_direction = Action.NORTH.name
    global last_direction
    
    # Check if agent is in the next cell
    for move in preferred_moves:
        if move not in dangerous_geese_positions and move not in player_goose:
            # Get direction of new move
            temp_direction = make_move(player_head, move, rows, columns)
            
            # Check if direction is not going 180 degees to current direction
            if temp_direction != opposite_direction[last_direction]:
                new_direction = temp_direction
                break
    
    # Update direction of last move 
    last_direction = new_direction
    
    return new_direction

In [ ]:
!pip install kaggle_environments==1.7.6

In [ ]:
from kaggle_environments import evaluate, make, utils
env = make("hungry_geese")

env.reset()
env.run(
    [
        "agent.py", 
        "greedy_agent.py",
        "agent.py", 
        "greedy_agent.py"
        
    ],
)
env.render(mode="ipython", width=800, height=700)